In [1]:
%load_ext autoreload
%autoreload 2

# Submesh Generation v2

Whereas my previous implementation of submeshes using meshview created a new mesh for each of the one-dimensional submeshes, I would rather want to only create a single lower-dimensional mesh for different types of boundary conditions.

The steps required will be:
* Load mesh, subdomain info, and facet info as usual.
* Create a "characteristic meshfunction" for facets labeled by a specific set of values.
* Create a submesh for each of the subdomains.
* Create a facetmap on each subdomain and map the boundary tags onto this facetmap.
    * Verify correctness using VTK Plotter
* Create a cellmap on the interface-mesh.
    * Verify cerrectness using VTK Plotter
* Attempt to define a characteristic equation on parts of the interface mesh.
    * Verify using VTK Plotter

In [2]:
from pathlib import Path

import h5py

import numpy as np
import matplotlib.pyplot as plt
from dolfin import *

from neuralthreesome.meshprocessing import geo2hdf, hdf2fenics
from neuralthreesome.subdomains import create_submesh, tag_interface_subdomains, tag_subdomain_facets
from neuralthreesome.characteristic_equation import Characteristic

In [3]:
# Path to any results.
resultspath = Path('../results/')
resultspath.mkdir(parents=True, exist_ok=True)

# Path to gmsh-file, and where to output the resulting meshfile
geofile = "../resources/test-domain.geo"
meshfile = "../mesh/test.h5"

# Create mesh from the gmsh-file
geo2hdf(geofile, meshfile, cleanup=False)

# Load the mesh and the MeshFunctions tagging the different subdomains.
mesh, subdomain_tags, boundary_tags = hdf2fenics(meshfile)

iface = create_submesh(mesh, boundary_tags, [3, 4])
sub1 = create_submesh(mesh, subdomain_tags, [1])
sub2 = create_submesh(mesh, subdomain_tags, [2])
interface_tags = tag_interface_subdomains(iface, boundary_tags)
subdomain_boundaries = tag_subdomain_facets(mesh, sub1, boundary_tags, [1, 3, 4])

In [4]:
V = FunctionSpace(sub1, "CG", 1)
W = FunctionSpace(iface, "CG", 1)
chi = Characteristic(interface_tags, 3)
chi = interpolate(chi, W)
w = Expression("x[1] * x[1]", degree=1)

u = Function(W)
u += chi * w

file = File(str("newfile.pvd"))
file << subdomain_boundaries
file << interface_tags
file << project(u, W)